In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import CustomObjectScope
from unet import unet
import numpy as np
import pandas as pd
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" # tf log messages suppression
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score
from tqdm import tqdm
from metrics import dice_loss, dice_coefficient
from train import load_dataset

In [2]:
# Global parameters
H = 256
W = 256

In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [4]:
def save_results(image, mask, y_pred, save_image_path): # Make sure dimensions are equal
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)
    
    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred * 255.0

    line = np.ones((H, 2, 3)) * 255.0
    
    concat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    concat_images = np.uint8(concat_images)
    cv2.imwrite(save_image_path, concat_images)

In [5]:
if __name__ == "__main__":
    
    # Seeding
    np.random.seed(42)
    tf.random.set_seed(42)
    
    # Directory for storing files
    create_dir("results")

    # Load the model
    with CustomObjectScope({"dice_coefficient":dice_coefficient, "dice_loss":dice_loss}):
        model = tf.keras.models.load_model(os.path.join("files","model.keras"))

    # Dataset
    dataset_path = r"C:\Users\batvi\MRI-tumor-segmentation\Dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

    # Prediction and Evaluation
    SCORE = []
    for x, y in tqdm(zip(test_x,test_y), total = len(test_x)):
        
        # Extract the name
        name = x.split("\\")[-1]
        
        # Read the image
        image = cv2.imread(x, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (W, H))
        x = image / 255.0                 # (h, w, 3)
        x = np.expand_dims(x, axis=0) # (1, h, w, 3)
        
        # Read the image
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (W, H))
        
        # Prediction
        y_pred = model.predict(x, verbose = 0)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred >= 0.5
        y_pred = y_pred.astype(np.int32)
        
        # Save the results
        save_image_path = os.path.join("results", name)
        save_results(image, mask, y_pred, save_image_path)
        
        # Flatten the array
        mask = (mask >= 0.5).astype(np.int32).flatten()
        y_pred = y_pred.flatten()
        
        #Calculate metrics
        f1_value = f1_score(mask, y_pred, labels=[0,1], average = "binary")
        jac_value = jaccard_score(mask, y_pred, labels=[0,1], average = "binary")
        recall_value = recall_score(mask, y_pred, labels=[0,1], average = "binary")
        precision_value = precision_score(mask, y_pred, labels=[0,1], average = "binary")
        SCORE.append([name, f1_value, jac_value, recall_value, precision_value])
    # Metrics
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"F1: {score[0]:0.5f}")
    print(f"Jaccard: {score[1]:0.5f}")
    print(f"Recall: {score[2]:0.5f}")
    print(f"Precision: {score[3]:0.5f}")

    df = pd.DataFrame(SCORE, columns=["Image", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

  3%|▎         | 21/612 [00:11<05:16,  1.87it/s]C:\Users\batvi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
  5%|▌         | 31/612 [00:17<05:36,  1.73it/s]C:\Users\batvi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
  6%|▌         | 35/612 [00:19<05:10,  1.86it/s]C:\Users\batvi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being

F1: 0.71734
Jaccard: 0.63460
Recall: 0.70162
Precision: 0.79235
